In [19]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier


from Modules import Modules
from Baseline import Baseline
from FeatureEngineering import FeatureEngineering
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

OK



In [20]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

In [3]:
def ligthgbm_feature_selection(dataframe, index_cols, auc_limit=0.7):
        dataframe = dataframe.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '_', x))
        clf = LGBMClassifier(random_state=0)
        train_df = dataframe[dataframe['TARGET'].notnull()]
        train_df_X = train_df.drop('TARGET', axis=1)
        train_df_y = train_df['TARGET']
        train_columns = [col for col in train_df_X.columns if col not in index_cols]

        max_auc_score = 1
        best_cols = []
        while max_auc_score > auc_limit:
            train_columns = [col for col in train_columns if col not in best_cols]
            clf.fit(train_df_X[train_columns], train_df_y)
            feats_imp = pd.Series(clf.feature_importances_, index=train_columns)
            max_auc_score = roc_auc_score(train_df_y, clf.predict_proba(train_df_X[train_columns])[:, 1])
            best_cols = feats_imp[feats_imp > 0].index.tolist()

        dataframe.drop(train_columns, axis=1, inplace=True)
        return dataframe

In [4]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [5]:
path = './home-credit-default-risk/'

## ファイル読み込み

In [6]:
bureau = FeatureEngineering.bureau_and_balance(path)
bureau.head()

Memory usage of dataframe: 4114.69 MB
Memory usage after optimization: 1189.22 MB
Decreased by 71.1%


,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [7]:
bureau = bureau.replace([np.inf, -np.inf], np.nan)
#bureau = bureau.fillna(bureau.max() + (bureau.max() * 0.4))
bureau.head()

,BURO_SK_ID_CURR_MIN,BURO_SK_ID_CURR_MAX,BURO_SK_ID_CURR_SIZE,BURO_SK_ID_CURR_MEAN,BURO_SK_ID_CURR_VAR,BURO_SK_ID_CURR_SUM,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_SIZE,BURO_DAYS_CREDIT_MEAN,...,CLOSED_Last_status_3_MEAN_SIZE,CLOSED_Last_status_3_MEAN_MEAN,CLOSED_Last_status_3_MEAN_VAR,CLOSED_Last_status_3_MEAN_SUM,CLOSED_Last_status_4_MEAN_MIN,CLOSED_Last_status_4_MEAN_MAX,CLOSED_Last_status_4_MEAN_SIZE,CLOSED_Last_status_4_MEAN_MEAN,CLOSED_Last_status_4_MEAN_VAR,CLOSED_Last_status_4_MEAN_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,7,100001.0,0.0,700007,-1572,-49,7,-735.000,...,4.0,0.000000,0.000000,0.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0
100002,100002,100002,8,100002.0,0.0,800016,-1437,-103,8,-874.000,...,6.0,0.000000,0.000000,0.0,0.000000,0.000000,6.0,0.000000,0.000000,0.0
100003,100003,100003,4,100003.0,0.0,400012,-2586,-606,4,-1401.000,...,3.0,1.400391,0.700195,0.0,1.400391,1.400391,3.0,1.400391,0.700195,0.0
100004,100004,100004,2,100004.0,0.0,200008,-1326,-408,2,-867.000,...,2.0,1.400391,0.700195,0.0,1.400391,1.400391,2.0,1.400391,0.700195,0.0
100005,100005,100005,3,100005.0,0.0,300015,-373,-62,3,-190.625,...,1.0,0.000000,0.700195,0.0,0.000000,0.000000,1.0,0.000000,0.700195,0.0


In [8]:
previous = FeatureEngineering.previous_application(path)
previous.head()

Memory usage of dataframe: 2434.34 MB
Memory usage after optimization: 818.55 MB
Decreased by 66.4%


,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [9]:
previous = previous.replace([np.inf, -np.inf], np.nan)
#previous = previous.fillna(previous.max() + (previous.max() * 0.4))
previous.head()

,PREV_SK_ID_PREV_MIN,PREV_SK_ID_PREV_MAX,PREV_SK_ID_PREV_SIZE,PREV_SK_ID_PREV_MEAN,PREV_SK_ID_PREV_VAR,PREV_SK_ID_PREV_SUM,PREV_SK_ID_CURR_MIN,PREV_SK_ID_CURR_MAX,PREV_SK_ID_CURR_SIZE,PREV_SK_ID_CURR_MEAN,...,REFUSED_PRODUCT_COMBINATION_POS_other_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_X_Sell_MEAN,REFUSED_PRODUCT_COMBINATION_POS_mobile_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Card_Street_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_with_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__low_MEAN,REFUSED_PRODUCT_COMBINATION_POS_industry_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_Cash_Street__middle_MEAN,REFUSED_PRODUCT_COMBINATION_POS_others_without_interest_MEAN,REFUSED_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1369693,1,1369693.00,2.367471e+12,1369693,100001,100001,1,100001.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100002,1038818,1038818,1,1038818.00,2.367471e+12,1038818,100002,100002,1,100002.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100003,1810518,2636178,3,2281150.25,1.804519e+11,6843451,100003,100003,3,100003.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100004,1564014,1564014,1,1564014.00,2.367471e+12,1564014,100004,100004,1,100004.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391
100005,1857999,2495675,2,2176837.00,2.033153e+11,4353674,100005,100005,2,100005.0,...,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391,1.400391


In [10]:
pos = FeatureEngineering.pos_cash(path)
pos.head()

Memory usage of dataframe: 147.95 MB
Memory usage after optimization: 46.96 MB
Decreased by 68.3%


,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [11]:
pos = pos.replace([np.inf, -np.inf], np.nan)
#pos = pos.fillna(pos.max() + (pos.max() * 0.4))
pos.head()

,POS_SK_ID_PREV_MIN,POS_SK_ID_PREV_MAX,POS_SK_ID_PREV_SIZE,POS_SK_ID_PREV_MEAN,POS_SK_ID_PREV_VAR,POS_SK_ID_PREV_SUM,POS_SK_ID_CURR_MIN,POS_SK_ID_CURR_MAX,POS_SK_ID_CURR_SIZE,POS_SK_ID_CURR_MEAN,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,9,1584044.50,6.461239e+10,14256401,100001,100001,9,100001.0,...,0.777832,0.222168,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,9
100002,1038818,1038818,19,1038818.00,0.000000e+00,19737542,100002,100002,19,100002.0,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19
100003,1810518,2636178,28,2297665.25,1.086316e+11,64334628,100003,100003,28,100003.0,...,0.928711,0.071411,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,28
100004,1564014,1564014,4,1564014.00,0.000000e+00,6256056,100004,100004,4,100004.0,...,0.750000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4
100005,2495675,2495675,11,2495675.00,0.000000e+00,27452425,100005,100005,11,100005.0,...,0.818359,0.090881,0.090881,0.0,0.0,0.0,0.0,0.0,0.0,11


In [12]:
installments = FeatureEngineering.installments_payments(path)
installments.head()

Memory usage of dataframe: 199.49 MB
Memory usage after optimization: 73.84 MB
Decreased by 63.0%


,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [13]:
installments = installments.replace([np.inf, -np.inf], np.nan)
#installments = installments.fillna(installments.max() + (installments.max() * 0.4))
installments.head()

,INS_SK_ID_PREV_MIN,INS_SK_ID_PREV_MAX,INS_SK_ID_PREV_SIZE,INS_SK_ID_PREV_MEAN,INS_SK_ID_PREV_VAR,INS_SK_ID_PREV_SUM,INS_SK_ID_CURR_MIN,INS_SK_ID_CURR_MAX,INS_SK_ID_CURR_SIZE,INS_SK_ID_CURR_MEAN,...,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_MEAN,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_VAR,INS_ins_AMT_INSTALMENT_AMT_PAYMENT_SUM,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MIN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MAX,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SIZE,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_MEAN,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_VAR,INS_ins_AMT_PAYMENT_div_AMT_INSTALMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1369693,1851984,7,1576389.125,6.645846e+10,11034724,100001,100001,7,100001.0,...,0.0,0.0,0.0,1.0,1.0,7,1.0,0.0,7.0,7
100002,1038818,1038818,19,1038818.000,0.000000e+00,19737542,100002,100002,19,100002.0,...,0.0,0.0,0.0,1.0,1.0,19,1.0,0.0,19.0,19
100003,1810518,2636178,25,2290070.250,1.027131e+11,57251754,100003,100003,25,100003.0,...,0.0,0.0,0.0,1.0,1.0,25,1.0,0.0,25.0,25
100004,1564014,1564014,3,1564014.000,0.000000e+00,4692042,100004,100004,3,100004.0,...,0.0,0.0,0.0,1.0,1.0,3,1.0,0.0,3.0,3
100005,2495675,2495675,9,2495675.000,0.000000e+00,22461075,100005,100005,9,100005.0,...,0.0,0.0,0.0,1.0,1.0,9,1.0,0.0,9.0,9


In [14]:
credit = FeatureEngineering.credit_card_balance(path)
credit.head()

Memory usage of dataframe: 173.03 MB
Memory usage after optimization: 66.37 MB
Decreased by 61.6%


,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [15]:
credit = credit.replace([np.inf, -np.inf], np.nan)
#credit = credit.fillna(credit.max() + (credit.max() * 0.4))
credit.head()

,CARD_SK_ID_PREV_MIN,CARD_SK_ID_PREV_MAX,CARD_SK_ID_PREV_SIZE,CARD_SK_ID_PREV_MEAN,CARD_SK_ID_PREV_VAR,CARD_SK_ID_PREV_SUM,CARD_SK_ID_CURR_MIN,CARD_SK_ID_CURR_MAX,CARD_SK_ID_CURR_SIZE,CARD_SK_ID_CURR_MEAN,...,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_VAR,CARD_card_AMT_DRAWINGS_CURRENT__AMT_DRAWINGS_POS_CURRENT_SUM,CARD_NAME_CONTRACT_STATUS_Active_MEAN,CARD_NAME_CONTRACT_STATUS_Completed_MEAN,CARD_NAME_CONTRACT_STATUS_Demand_MEAN,CARD_NAME_CONTRACT_STATUS_Signed_MEAN,CARD_NAME_CONTRACT_STATUS_Sent_proposal_MEAN,CARD_NAME_CONTRACT_STATUS_Refused_MEAN,CARD_NAME_CONTRACT_STATUS_Approved_MEAN,CARD_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396,1489396,6,1489396.0,0.0,8936376,100006,100006,6,100006.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6
100011,1843384,1843384,74,1843384.0,0.0,136410416,100011,100011,74,100011.0,...,4.378378e+08,180000.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74
100013,2038692,2038692,96,2038692.0,0.0,195714432,100013,100013,96,100013.0,...,8.249688e+08,571500.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,96
100021,2594025,2594025,17,2594025.0,0.0,44098425,100021,100021,17,100021.0,...,8.505000e+11,0.0,0.411865,0.588379,0.0,0.0,0.0,0.0,0.0,17
100023,1499902,1499902,8,1499902.0,0.0,11999216,100023,100023,8,100023.0,...,8.505000e+11,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8


In [16]:
df = FeatureEngineering.application_train_test(path, 'application_train.csv')
df.head()

Memory usage of dataframe: 404.30 MB
Memory usage after optimization: 169.87 MB
Decreased by 58.0%


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,app_AMT_INCOME_TOTAL_div_CNT_CHILDREN,app_most_popular_AMT_GOODS_PRICE,app_popular_AMT_GOODS_PRICE,app_OWN_CAR_AGE_div_DAYS_BIRTH,app_OWN_CAR_AGE_div_DAYS_EMPLOYED,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH,app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED,app_DAYS_EMPLOYED_PERC,app_DAYS_EMPLOYED_div_DAYS_BIRTH,app_CNT_CHILDREN_div_CNT_FAM_MEMBERS
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,202500.0,0,0,NaN,NaN,0.119873,1.780273,8824.0,0.067322,0.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,270000.0,0,0,NaN,NaN,0.049377,0.696777,15576.0,0.070862,0.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,67500.0,0,0,-0.001365,-0.11554,0.042786,3.623047,18816.0,0.011810,0.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,135000.0,0,0,NaN,NaN,0.032471,0.203003,15968.0,0.159912,0.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,121500.0,0,0,NaN,NaN,0.055481,0.364014,16896.0,0.152466,0.0


### データフレームを結合

In [17]:
### データフレームをマージ
df = pd.merge(df, bureau, on='SK_ID_CURR', how='left')
df = pd.merge(df, installments, on='SK_ID_CURR', how='left')
df = pd.merge(df, previous, on='SK_ID_CURR', how='left')
df = pd.merge(df, credit, on='SK_ID_CURR', how='left')
df = pd.merge(df, pos, on='SK_ID_CURR', how='left')
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Returned_to_the_store_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_Amortized_debt_MEAN,POS_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,19.0
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.928711,0.071411,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,28.0
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.750000,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.0
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.856934,0.095215,0.000000,0.0,0.047607,0.0,0.0,0.0,0.0,21.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.939453,0.045441,0.015152,0.0,0.000000,0.0,0.0,0.0,0.0,66.0


In [21]:
df = FeatureEngineering.clean_data(df)
df.head()

After removing empty features there are 3279 features


AttributeError: 'FeatureEngineering' object has no attribute 'ranksums'

### 欠損値補完

In [ ]:
### 処理対象の列名を配列化
replace_columns = list(df.select_dtypes(include='number').columns)
replace_columns.remove('SK_ID_CURR')
replace_columns.remove('TARGET')

In [ ]:
#for column in replace_columns:
#    df[column].fillna(df[column].max() + (df[column].max() * 0.4))
#
#df.head()

### 変数選択

In [ ]:
### 変数選択
df = FeatureEngineering2.data_post_processing(df)
df.head()

### 標準化・対数変換

In [ ]:
### 処理対象の列名を配列化
encoding_columns = list(df.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [ ]:
### indexを設定
df = df.set_index('SK_ID_CURR')
df.head()

In [ ]:
### 標準化
scaler = StandardScaler()

for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    scaler.fit(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])
        
df.head()

In [ ]:
### indexを削除
df = df.reset_index()
df.head()

In [ ]:
### Yao-Jhonson変換
### 学習データに基づいてYao-Johnson変換を定義
#for column in encoding_columns:
#    pt = PowerTransformer(method = 'yeo-johnson')
#    pt.fit(df[column].copy())

    ### 変換後のデータで各列を置換
#    df[column] = pt.transform(df[column])

### train/testを分割

In [ ]:
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

In [ ]:
del df
del bureau
del installments
del previous
del credit
del pos

gc.collect()

In [ ]:
app_test.drop('TARGET', axis=1, inplace=True)
print(app_test.shape)
app_test.head()

In [ ]:
app_train = app_train.replace([np.inf, -np.inf], np.nan)
#app_train = app_train.fillna(app_train.max() + (app_train.max() * 0.4))
app_train.head()

In [ ]:
app_test = app_test.replace([np.inf, -np.inf], np.nan)
#app_test = app_test.fillna(app_test.max() + (app_test.max() * 0.4))
app_test.head()

## CSVファイルを出力

In [ ]:
df_train = app_train.copy()
df_train.head()

In [ ]:
df_train.to_csv(
    path_or_buf="./home-credit-default-risk/exports/df_train.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [ ]:
df_test = app_test.copy()
df_test.head()

In [ ]:
df_test.to_csv(
    path_or_buf="./home-credit-default-risk/exports/df_test.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [ ]:
del app_train
del app_test
gc.collect()

## CSVファイルを読み込み

## 前処理

In [ ]:
for col in df_train.columns:
    df_train[col] = df_train[col].replace('\s', '', regex=True)
    df_train[col] = df_train[col].replace(':', '', regex=True)
    df_train[col] = df_train[col].replace('/', '', regex=True)
    df_train[col] = df_train[col].replace(r'\(', r'', regex=True)
    df_train[col] = df_train[col].replace(r'\)', r'', regex=True)
    df_train[col] = df_train[col].replace(r'\+', r'', regex=True)
    df_train[col] = df_train[col].replace(r'\,', r'', regex=True)
    print(col)

In [ ]:
df_train = df_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df_train.head()

In [ ]:
for col in df_test.columns:
    df_test[col] = df_test[col].replace('\s', '', regex=True)
    df_test[col] = df_test[col].replace(':', '', regex=True)
    df_test[col] = df_test[col].replace('/', '', regex=True)
    df_test[col] = df_test[col].replace(r'\(', r'', regex=True)
    df_test[col] = df_test[col].replace(r'\)', r'', regex=True)
    df_test[col] = df_test[col].replace(r'\+', r'', regex=True)
    df_test[col] = df_test[col].replace(r'\,', r'', regex=True)
    print(col)

In [ ]:
df_test = df_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df_test.head()

In [ ]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

## 学習

### 学習(train)

In [ ]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [ ]:
baseline = Baseline('SK_ID_CURR')

"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

In [ ]:
train_columns = list(df_train.columns.sort_values(ascending=False))

for col in train_columns:
    print(col)

In [ ]:
test_columns = list(df_test.columns.sort_values(ascending=False))

for col in test_columns:
    print(col)

In [ ]:
imp.sort_values("imp", ascending=False)[:20]

### 学習(test)

In [ ]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [ ]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

In [ ]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSource1-01.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

## パラメータ・チューニング

In [ ]:
baseline = Baseline('SK_ID_CURR')

sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

In [ ]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

In [ ]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

In [ ]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

In [ ]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_featureEngineeringOtherSource2-1.csv", index=None)

In [ ]:
del df_train
del df_test
gc.collect()